In [4]:
# Initial Setting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from pykrx import stock

In [6]:
df = stock.get_market_ohlcv_by_ticker("20210122")
print(df.head())

            시가      고가      저가      종가      거래량         거래대금    등락률
티커                                                                 
095570    4190    4245    4160    4210   216835    910274405   0.84
006840   25750   29550   25600   29100   727088  20462325950  12.57
027410    5020    5250    4955    5220  1547629   7990770515   4.19
282330  156500  156500  151500  152000    62510   9555364000  -2.56
138930    5720    5720    5610    5610  2094260  11808815750  -2.09


거래량 순으로 정렬:

In [8]:
sorted_df = df.sort_values(by=['거래량'], axis=0, ascending = False)
print(sorted_df.head(30))

           시가     고가     저가     종가       거래량           거래대금    등락률
티커                                                                
002630   2015   2390   1965   2200  68498831   150466894990  13.11
011300    932   1100    861   1025  43792816    44343776096  13.89
005930  89000  89700  86800  86800  30861661  2717635251520  -1.48
002880   1320   1370   1160   1240  28712924    35842590105  -1.20
033180   5800   5840   5320   5660  26652361   150598121230  -1.05
001360   9340  10150   9030  10150  24170798   236113458480   7.52
041650   4945   6100   4915   5060  22823315   124789874300   4.87
001440   1385   1475   1375   1445  20669239    29969417785   4.33
145210    655    736    651    672  20512271    14441141545   2.60
008420   3760   4220   3535   3555  19664658    78400904805  -2.60
010100   3790   4270   3405   3405  19659071    77906616815  -6.07
092220   3280   3290   3095   3145  18801798    60077417240  -6.40
024900   3170   4195   3035   3900  17602085    67036850590  1